In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType, LongType, DoubleType, FloatType, DataType, DateType, TimestampType,DecimalType
from pyspark.sql.functions import regexp_replace,row_number
from pyspark.sql.window import Window

In [ ]:
spark = (SparkSession
 .builder
 .appName("Fact")
 .getOrCreate())

In [ ]:
"""

Afther fetching all of the data and tranforming it, we read the data from the respective folder 

"""


profession_df = spark.read.option('header','True').csv('/home/ubuntu/Documents/Beautiful_Soap/Profession/*.csv')

state_city_df = spark.read.option('header','True').csv('/home/ubuntu/Documents/Beautiful_Soap/State/*.csv')

state_salary_df = spark.read.option('header','True').csv('/home/ubuntu/Documents/Beautiful_Soap/State_Salary_Deviation/*.csv')



In [ ]:
profession_df.show()

In [ ]:
state_city_df.show()

In [ ]:
state_salary_df.show()

In [ ]:
"""

Using Spark Sql to get data in one Fact Table so that this table can be used for visualization

"""


state_salary_df.createOrReplaceTempView('state_salary_table')
state_city_df.createOrReplaceTempView('state_city_table')
profession_df.createOrReplaceTempView('profession_table')

state_city_salary = spark.sql("SELECT t1.State,t1.Salary_Deviation,t2.City FROM state_salary_table t1 INNER JOIN state_city_table t2 ON t1.State LIKE t2.State")

#fact = state_salary_df.select('State','State_Salary_Deviation','City').join(state_city_df, state_salary_df.State == state_city_df.State).show()

In [ ]:
state_city_salary.createOrReplaceTempView('state_city_salary_table')

In [ ]:


final_df = spark.sql("SELECT t1.Profession, t1.Salary, t2.*,(Salary + (Salary*Salary_Deviation)/100 ) as Sal_Dev FROM profession_table as t1 CROSS JOIN state_city_salary_table as t2")

In [ ]:
final_df.write.format('csv').option('header','true').mode('overwrite').option('encoding','UTF-8').save('Fact')